In [ ]:
import pandas as pd
import gluestick as gs
import os
import json

from datetime import date
from dateutil.relativedelta import relativedelta

In [ ]:
# Get the sync_type (full_sync or incremental_sync)
sync_type = os.environ.get("SYNC_TYPE", "incremental_sync")

In [ ]:
INPUT_DIR = "./sync-output"
SNAPSHOT_DIR = "./snapshots"
OUTPUT_DIR = "./etl-output"

In [ ]:
# By default hotglue gets the last 2 periods + the current period
LOOKBACK_PERIOD = 2

# get tenant-config flags
config_path = f'{SNAPSHOT_DIR}/tenant-config.json'

config = {}
if os.path.exists(config_path):
    with open(config_path) as f:
        config = json.load(f)

In [ ]:
input = gs.Reader()

In [ ]:
# Get the report as a dataframe
stream = "GeneralLedgerCashReport"
gl_report = input.get(stream)

In [ ]:
# Get the snapped report data
gl_report_snap = gs.read_snapshots(stream, SNAPSHOT_DIR)

In [ ]:
def replace_beginning_balance_dates(df):
    beginning_balance_date = config.get("opening_balance_date")
    # check if there is a "Beginning Balance" in the Date column
    if "Beginning Balance" in df["Date"].unique():
        if beginning_balance_date is None:
            # filter out the smallest date from gl_report_snap and gl_report
            beginning_balance_date = df["Date"].min()
            # go the the last day of previous month
            beginning_balance_date = (pd.to_datetime(beginning_balance_date).replace(day=1) - pd.Timedelta(days=1)).strftime("%Y-%m-%d")

        if beginning_balance_date is not None:
            # Add "Beginning Balance" in Memo and replace "Beginning Balance" strings in Date column with the beginning balance date
            df.loc[df["Date"] == "Beginning Balance", "Memo"] = "Beginning Balance"
            df.loc[df["Date"] == "Beginning Balance", "Date"] = beginning_balance_date
    return df

In [ ]:
# If we have a pre-existing snapshot and new incremental data, we'll need to combine them together
if gl_report_snap is not None and gl_report is not None and sync_type == "incremental_sync":
    gl_report_snap = replace_beginning_balance_dates(gl_report_snap)
    # Ensure the Date column is a date
    gl_report_snap["Date"] = pd.to_datetime(gl_report_snap["Date"]).dt.date
    
    # Get today's date
    today = date.today()

    # Get the first day of the month LOOKBACK_PERIOD months ago
    first_day_three_months_ago = (today.replace(day=1) - relativedelta(months=LOOKBACK_PERIOD))
    
    # Drop the last 3 months of data (this is what the incremental sync brought in)
    print(f"Clearing snapped data newer than {first_day_three_months_ago}")
    gl_report_snap = gl_report_snap[gl_report_snap["Date"] < first_day_three_months_ago]
    
    # Combine with gl_report
    print("Combining the incremental data with the snapshot data")
    gl_report = pd.concat([gl_report_snap, gl_report])

In [ ]:
if gl_report is not None:
    gl_report = replace_beginning_balance_dates(gl_report)
    # If the Date column is a date, turn it back to str
    gl_report['Date'] = gl_report['Date'].astype("string")

    # NOTE: We are passing overwrite=True because we are already stitching the snapshot with the incremental data above
    gs.snapshot_records(gl_report, stream, SNAPSHOT_DIR, overwrite=True, use_csv=True)

In [ ]:
# Write the final output (a full GL Report)
gs.to_export(gl_report, stream, OUTPUT_DIR, export_format="csv")